In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
import cv2

In [27]:
from keras.preprocessing import image

In [28]:
# image_path_samples = '../train_camVid/*.png'
# image_path_labels = '../LabeledApproved_full/*.png'
image_path_samples = 'C:/Krishna/DataSets/cam_vid_semantic/train_samples/*.png'
image_path_labels = 'C:/Krishna/DataSets/cam_vid_semantic/train_labels/*.png'


In [29]:
import glob
filelist_samples = glob.glob(image_path_samples)
filelist_labels = glob.glob(image_path_labels)



In [5]:
from PIL import Image

samples_array =np.array([np.array(image.load_img(fname, target_size=(256, 256))) for fname in filelist_samples])
labels_array = np.array([np.array(image.load_img(fname, target_size=(256, 256))) for fname in filelist_labels])

print(samples_array.shape,labels_array.shape)

(701, 256, 256, 3) (701, 256, 256, 3)


In [40]:
#from keras.utils import to_categorical
from PIL import Image

np.unique(np.array(Image.open(filelist_labels[500])))

array([  0,  64, 128, 192], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split

train_samples_array,test_samples_array,train_labels_array,test_labels_array = \
train_test_split(samples_array,labels_array,test_size=0.33, random_state=42)

In [ ]:
print("train Samples size {} and test samples size {}".format(train_samples_array.shape,test_samples_array.shape))

In [ ]:
from matplotlib.pyplot import imshow
%matplotlib inline

imshow(Image.fromarray(np.squeeze(train_samples_array[20])))


In [ ]:
imshow(Image.fromarray(np.squeeze(train_labels_array[20])))

In [ ]:
def pretrained_path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(256, 256))
    # convert PIL.Image.Image type to 3D tensor with shape (512, 512, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 512, 512, 3) and return 4D tensor
    x = np.expand_dims(x, axis=0)

    return x

In [ ]:
## normalize tensors

train_samples_array = train_samples_array.astype('float32')/255
test_samples_array = test_samples_array.astype('float32')/255 
train_labels_array = train_labels_array.astype('float32')/255
test_labels_array = test_labels_array.astype('float32') /255


In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint


In [ ]:
## Model for FCNNs with skip connections

img_input = Input(shape=(256,256,3))
conv1 = Conv2D(64,(3,3),activation= 'relu',padding='same')(img_input)
pool1 = MaxPool2D(pool_size=(2,2),padding='same')(conv1)

conv2 = Conv2D(128,(3,3),activation= 'relu',padding='same')(pool1)
pool2 = MaxPool2D(pool_size=(2,2),padding='same')(conv2)

conv3 = Conv2D(256,(3,3),activation= 'relu',padding='same')(pool2)
drop3 = Dropout(0.3)(conv3)
pool3 = MaxPool2D(pool_size=(2,2),padding='same')(drop3)

conv4 = Conv2D(512,(3,3),activation= 'relu',padding='same')(pool3)
drop4 = Dropout(0.3)(conv4)

# up_sample4 = UpSampling2D((2,2))(drop4)
# conv_transpose4 = Conv2DTranspose(512,(3,3),activation='relu',padding='same')(up_sample4)
# skip_connected_4 = Add()([conv_transpose4,drop4])

up_sample3 = UpSampling2D((2,2))(drop4)
conv_transpose3 = Conv2DTranspose(256,(3,3),activation='relu',padding='same')(up_sample3)
skip_connected_3 = Add()([conv_transpose3,drop3])

up_sample2 = UpSampling2D((2,2))(skip_connected_3)
conv_transpose2 = Conv2DTranspose(128,(3,3),activation='relu',padding='same')(up_sample2)
skip_connected_2 = Add()([conv_transpose2,conv2])

up_sample1 = UpSampling2D((2,2))(skip_connected_2)
conv_transpose1 = Conv2DTranspose(64,(3,3),activation='relu',padding='same')(up_sample1)
out_layer = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(conv_transpose1)

model = Model(input = img_input,output = out_layer)

model.summary()


In [ ]:
model.compile(optimizer = 'sgd', loss = 'mse', metrics = ['accuracy'])

In [ ]:
check_pointer = ModelCheckpoint('best_weights/best_weights.hdf5',verbose=1,save_best_only=True)
history = model.fit(train_samples_array,train_labels_array,epochs=5,verbose=1,validation_split=0.1, batch_size=5,callbacks=[check_pointer])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# model.fit(train_samples_array,train_labels_array,epochs=5,verbose=2,batch_size=5,validation_split=0.2,callbacks=[check_pointer])

In [ ]:
test_metrics = model.evaluate(test_samples_array,test_labels_array)

print(test_metrics)

In [ ]:
pred_array = model.predict(np.expand_dims(train_samples_array[20],axis=0))
label_array =  train_labels_array[20]

In [ ]:
imshow(Image.fromarray(np.squeeze((pred_array*255).astype('uint8'))))


In [ ]:
imshow(Image.fromarray(np.squeeze((label_array*255).astype('uint8'))))